In [6]:
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import numpy as np
import chainer
import numpy as np
from sklearn.datasets import load_sample_image
import cv2
 
basedir = './src/cnn/images'

CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]
 
cifartrain, cifartest=tf.keras.datasets.cifar100.load_data(label_mode='fine')
cifartr_img=cifartrain[0]
cifartr_lbl=cifartrain[1]
cifarts_img=cifartest[0]
cifarts_lbl=cifartest[1]




config = tf.ConfigProto()
config.gpu_options.allow_growth = True

class CNN():
    def __init__(self,w,h,ch,num):
        #Assigns a space for future use
        self.x = tf.placeholder(dtype=tf.float32, shape=(None,h,w,ch))

        #First convolutionary layer, it attempts to find patterns
        conv0=tf.layers.conv2d(self.x, filters=300, kernel_size=[1, 1],activation=tf.nn.relu)

        #First pool layer, averages the imgae, lowers res
        pool0=tf.layers.max_pooling2d(conv0,pool_size=[2, 2],strides=2)

        #Second convolutionary layer, attemps to find even more patterns
        conv1=tf.layers.conv2d(pool0, filters=300, kernel_size=[1, 1],activation=tf.nn.relu)

        #second pool layer, averages the imgae, lowers res
        pool1=tf.layers.max_pooling2d(conv1,pool_size=[2, 2],strides=2)

        #First flat layer, Compresses everything into an array
        flat0=tf.layers.flatten(pool1)

        #First dense layer, decides tha final classification of the image
        dense0=tf.layers.dense(flat0,1024,activation=tf.nn.relu)

        #Drop layer, gets rid of useless neurons
        drop0=tf.layers.dropout(dense0,rate=0.4,training=True)

        #Second dense layer, thsi layer is the final layer
        out0=tf.layers.dense(drop0,num)

        #It fins the maximunand sets the rest to 0
        self.choice = tf.argmax(out0, axis = 1)

        #Probability based on weight
        self.prob = tf.nn.softmax(out0)
        
        self.labels = tf.placeholder(dtype=tf.float32, name='labels')
        
        self.accuracy, self.accuracy_op =  tf.metrics.accuracy(self.labels, self.choice)
        
        onehotlabel = tf.one_hot(indices=tf.cast(self.labels, dtype=tf.int32), depth=num)
        
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=onehotlabel,logits=out0)
        
        #Trains rate
        opt=tf.train.GradientDescentOptimizer(0.002)
         
        #Train fuction to get better and learn
        self.train = opt.minimize(self.loss, global_step=tf.train.get_global_step())
        

In [7]:
tf.reset_default_graph()

trainingsteps=100000
batchsize=16
imgh=32
imgw=32
imgch=3
trainingdata0=cifartr_img
traininglabel0=cifartr_lbl

cnn=CNN(imgh,imgw,imgch,100)

trainingdata1 = trainingdata0.reshape(-1,imgw,imgh,imgch)
dataset = tf.data.Dataset.from_tensor_slices((trainingdata1,traininglabel0))
dataset =dataset.shuffle(buffer_size=traininglabel0.shape[0])
dataset = dataset.batch(batchsize)
dataset = dataset.repeat()
dataset_iterator = dataset.make_initializable_iterator()
next_element = dataset_iterator.get_next()

saver=tf.train.Saver()

with tf.Session(config=config) as sess:
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    saver.restore(sess, '/home/student/Desktop/edu/CNNsaves/cifar100.ckpt')
    sess.run(dataset_iterator.initializer)
    for i in range(trainingsteps):
        current=sess.run(next_element)
        batch_inputs = current [0]
        batch_labels = current [1]
        print(i,sess.run((cnn.train, cnn.accuracy_op), feed_dict={cnn.x:batch_inputs, cnn.labels:batch_labels}))
        if i % 5000 == 0:
            print('Attempting to save the CNN!')
            sdir = saver.save(sess, '/home/student/Desktop/edu/CNNsaves/cifar100.ckpt')
            print('Complete!')
    print(sess.run(cnn.accuracy))
    print('Attempting to save the CNN!')
    sdir = saver.save(sess, '/home/student/Desktop/edu/CNNsaves/cifar100.ckpt')
    print('Complete!')

INFO:tensorflow:Restoring parameters from /home/student/Desktop/edu/CNNsaves/cifar100.ckpt
0 (None, 0.1875)
Attempting to save the CNN!
Complete!
1 (None, 0.125)
2 (None, 0.10416666)
3 (None, 0.109375)
4 (None, 0.1)
5 (None, 0.10416666)
6 (None, 0.098214284)
7 (None, 0.09375)
8 (None, 0.097222224)
9 (None, 0.09375)
10 (None, 0.090909094)
11 (None, 0.098958336)
12 (None, 0.10096154)
13 (None, 0.11607143)
14 (None, 0.12083333)
15 (None, 0.12109375)
16 (None, 0.12867647)
17 (None, 0.125)
18 (None, 0.125)
19 (None, 0.125)
20 (None, 0.12797619)
21 (None, 0.13068181)
22 (None, 0.13043478)
23 (None, 0.13020833)
24 (None, 0.13249999)
25 (None, 0.1298077)
26 (None, 0.12731482)
27 (None, 0.12723215)
28 (None, 0.125)
29 (None, 0.12708333)
30 (None, 0.125)
31 (None, 0.125)
32 (None, 0.125)
33 (None, 0.12316176)
34 (None, 0.125)
35 (None, 0.12326389)
36 (None, 0.12162162)
37 (None, 0.125)
38 (None, 0.12179487)
39 (None, 0.1203125)
40 (None, 0.11890244)
41 (None, 0.11904762)
42 (None, 0.11918604)
43

KeyboardInterrupt: 

In [8]:
testimg=cifarts_img
print(testimg.shape)
testimg=testimg.reshape(10000,32,32,3)

(10000, 32, 32, 3)


In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    saver.restore(sess, '/home/student/Desktop/edu/CNNsaves/cifar100.ckpt')
    guess=sess.run(cnn.choice, feed_dict={cnn.x:testimg})
    test = testimg[0]
    test=test.reshape(32,32)
    plt.imshow(test)
    plt.show()
    guessing=str(guess[0])
    print(guessing, CIFAR100_LABELS_LIST)
    

INFO:tensorflow:Restoring parameters from /home/student/Desktop/edu/CNNsaves/cifar100.ckpt


ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,32,300]
	 [[Node: conv2d/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_Placeholder_0_0, conv2d/kernel/read)]]

Caused by op 'conv2d/Conv2D', defined at:
  File "/home/student/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/student/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/student/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/student/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home